In [ ]:
import pandas as pd
from sodapy import Socrata
import numpy as np
import sqlite3

# Get information from Cook County Sentencing Data Database

In [ ]:
# Call API
client = Socrata("datacatalog.cookcountyil.gov", None)

In [ ]:
# Retrive Data
results = client.get("tg8v-tm6u", limit=300000)

In [ ]:
# Generate DataFrame
original_data = pd.DataFrame.from_records(results)

# Explore data
list(original_data.columns.values)

In [ ]:
data = original_data.copy()

In [ ]:
# Explore values in columns
data.count()

In [ ]:
# Delete columns that we don't need
data = data.drop([
    'charge_disposition_reason',
    'unit',
    'disposition_charged_chapter',
    'disposition_charged_act',
    'disposition_charged_section',
    'incident_end_date',
    'law_enforcement_agency',
    'disposition_charged_aoic',
    'incident_city',
    'arraignment_date',
    'felony_review_date',
    'felony_review_result',
    'received_date',
    'disposition_date',
    'charge_id',
    'charge_version_id',
    'sentence_judge',
    'sentence_phase',
    'offense_category',
    'commitment_type'
],axis=1)

In [ ]:
# Latest offense_category data
data = data.rename(columns={'updated_offense_category':'offense_category'})

In [ ]:
data.count()

# Cleaning Process

## Filter data after 2000 & before 2019

In [ ]:
# Create a copy of the Sentence Date to filter it
data['date_year'] = data['sentence_date'].copy()

In [ ]:
# Select only the year
data['date_year'] = data['date_year'].replace(regex=['[0-9]*\/[0-9]*\/','\s[0-9]*\:[0-9]*\:[0-9]*\s[A-Z]*'], value='')

In [ ]:
# Change value type
data['date_year'] = data['date_year'].astype(int)

In [ ]:
# Confirm value type change
data.dtypes

In [ ]:
# Filter year
data = data[(data.date_year >= 2010) & (data.date_year <= 2020)]

In [ ]:
# Confirm values
data.describe()

## Filter only current sentence and primary charge

In [ ]:
# Drop False values for Current Sentence and Primary Charge
data = data.loc[data.current_sentence == True]
data = data.loc[data.primary_charge == True]

In [ ]:
data.count()

## Analize Offense Category and consolidate into related categories

In [ ]:
# Review all the information
data.offense_category.value_counts()

In [ ]:
data.offense_category.unique()

In [ ]:
# Consolidate Homicide category
data.offense_category = data.offense_category.replace(to_replace=['Attempt Homicide','Reckless Homicide'],value='Homicide')

In [ ]:
# Consolidate Offense Against Police Officers category
data.offense_category = data.offense_category.replace(to_replace=['Aggravated Assault Police Officer','Aggravated Assault Police Officer Firearm','Police Shooting','Impersonating Police Officer','Aggravated Battery Police Officer','Aggravated Battery Police Officer Firearm','Disarming Police Officer'],value='Offense Against Police Officers')

In [ ]:
# Consolidate Battery category
data.offense_category = data.offense_category.replace(to_replace=['Aggravated Battery','Aggravated Battery With A Firearm','Domestic Battery'],value='Battery')

In [ ]:
# Consolidate Robbery, Burglery and Theft category
data.offense_category = data.offense_category.replace(to_replace=['Aggravated Identity Theft','Aggravated Robbery','Aggravated Robbery BB Gun','Armed Robbery','Attempt Armed Robbery','Burglary','Identity Theft','Residential Burglary','Retail Theft','Robbery','Theft','Theft by Deception','Possession Of Burglary Tools'],value='Robbery/Burglary/Theft')

In [ ]:
# Consolidate Arson category
data.offense_category = data.offense_category.replace(to_replace=['Arson and Attempt Arson','Attempt Arson'],value='Arson')

In [ ]:
# Consolidate Firearms and Explosives category
data.offense_category = data.offense_category.replace(to_replace=['Aggravated Discharge Firearm','Armed Violence','Gun Running','Possession of Explosives','UUW - Unlawful Use of Weapon','Gun - Non UUW','Bomb Threat','Reckless Discharge of Firearm'],value='Firearms and Explosives')

In [ ]:
# Consolidate Motor Vehicles Offenses category
data.offense_category = data.offense_category.replace(to_replace=['Aggravated DUI','Attempt Vehicular Hijacking','DUI','Driving With Suspended Or Revoked License','Major Accidents','Possession of Stolen Motor Vehicle','Vehicular Hijacking','Vehicular Invasion'],value='Motor Vehicle Offenses')

In [ ]:
# Consolidate Judicial Process Violations category
data.offense_category = data.offense_category.replace(to_replace=['Communicating With Witness','Escape - Failure to Return','Obstructing Justice','Perjury','Tampering','Violate Bail Bond','Violation Order Of Protection'],value='Judicial Process Violations')

In [ ]:
# Consolidate Sex Offenses category
data.offense_category = data.offense_category.replace(to_replace=['Attempt Sex Crimes','Child Pornography','Failure to Register as a Sex Offender','Pandering','Prostitution','Sex Crimes','Violation of Sex Offender Registration'],value='Sex Offenses')

In [ ]:
# Consolidate Human Trafficking, Detention and Kidnapping category
data.offense_category = data.offense_category.replace(to_replace=['Child Abduction','Human Trafficking','Kidnapping','Unlawful Restraint'],value='Human Trafficking/Detention/Kidnapping')

In [ ]:
# Consolidate Trespassing category
data.offense_category = data.offense_category.replace(to_replace=['Home Invasion','Criminal Trespass To Residence'],value='Trespassing')

In [ ]:
# Consolidate Fraud and Deception category
data.offense_category = data.offense_category.replace(to_replace=['Credit Card Cases','Deceptive Practice','Forgery','Fraud','Fraudulent ID'],value='Fraud/Deception')

In [ ]:
# Consolidate Corruption category
data.offense_category = data.offense_category.replace(to_replace=['Intimidation','Official Misconduct','Bribery'],value='Corruption')

In [ ]:
# Consolidate Inside Penal Institutions category
data.offense_category = data.offense_category.replace(to_replace=['Possession of Contraband in Penal Institution','Possession of Shank in Penal Institution'],value='Inside Penal Institutions')

In [ ]:
# Consolidate Other Offense category
data.offense_category = data.offense_category.replace(to_replace=['Dog Fighting','Gambling','Failure To Pay Child Support','Compelling Gang Membership'],value='Other Offense')

In [ ]:
data.offense_category.value_counts()

## Cleaning Race columns

In [ ]:
# Review data
data.race.value_counts()

In [ ]:
# Consolidate Asian race
data.race = data.race.replace(to_replace='ASIAN',value='Asian')

In [ ]:
# Consolidate Hispanic/Latino race
data.race = data.race.replace(to_replace=['White [Hispanic or Latino]','HISPANIC','White/Black [Hispanic or Latino]'],value='Hispanic/Latino')

In [ ]:
data.race.value_counts()

## Cleaning Sentence Type

In [ ]:
# Review data
data.sentence_type.value_counts()

In [ ]:
# Consolidate Probation & Supervision sentence
data.sentence_type = data.sentence_type.replace(to_replace=['Probation','2nd Chance Probation','Supervision','Probation Terminated Unsatisfactorily','Probation Terminated Instanter','Probation Terminated Satisfactorily'],value='Probation/Supervision')

In [ ]:
# Consolidate Conditional sentence type
data.sentence_type = data.sentence_type.replace(to_replace='Conditional Release',value='Conditional Discharge')

In [ ]:
# Consolidate Incarceration sentence
data.sentence_type = data.sentence_type.replace(to_replace=['Jail','Prison'],value='Incarceration')

In [ ]:
data.sentence_type.value_counts()

## Cleaning Gender

In [ ]:
data.gender.value_counts()

In [ ]:
# Select only Male and Female genders due to size
data = data.loc[(data.gender == "Male") | (data.gender == "Female")]

In [ ]:
data.gender.value_counts()

## Cleaning Commitment Unit

In [ ]:
# Data Exploration
data.commitment_unit.value_counts()

In [ ]:
# Unify Weight units
data.commitment_unit = data.commitment_unit.replace(to_replace=['Pounds','Kilos','Ounces'],value='Weight')

In [ ]:
# Clean all non-numeric characters and transform value type to float
data.commitment_term = data.commitment_term.replace(to_replace='two',value=2)
data.commitment_term = data.commitment_term.replace(regex=['[a-z]*','\,','\`'], value='')
data.commitment_term = data.commitment_term.astype('float')

In [ ]:
# Review commitment unit values
data.commitment_unit.value_counts()

In [ ]:
# Change all years over 130 to 130 to mark natural life
data.loc[(data.commitment_unit == 'Year(s)') & (data.commitment_term > 129),['commitment_term']] = 130

In [ ]:
# Changes in Bootcamp Terms to Months
data.loc[(data.sentence_type == 'Cook County Boot Camp') & (data.commitment_unit == 'Term') & (data.commitment_term == 1),['commitment_term']] = 12
data.loc[(data.sentence_type == 'Cook County Boot Camp') & (data.commitment_unit == 'Term') & (data.commitment_term == 12),['commitment_unit']] = 'Months'
data.loc[(data.sentence_type == 'Cook County Boot Camp') & (data.commitment_unit == 'Term') & (data.commitment_term == 18),['commitment_unit']] = 'Months'

In [ ]:
# Change Death info to 130 years
data.loc[(data.sentence_type == 'Death'),['commitment_unit']] = 'Year(s)'
data.loc[(data.sentence_type == 'Death'),['commitment_term']] = 130

In [ ]:
# Create column with all values in months
def month_convert(row):
    if row ['commitment_unit'] == 'Months':
        return round(float(row['commitment_term']),2)
    if row["commitment_unit"] == "Year(s)":
        return round(int(row["commitment_term"]) * 12.0, 2)
    if row["commitment_unit"] == "Weeks":
       return round(float(row['commitment_term']) / 4, 2)
    if row["commitment_unit"] == "Days":
        return round(float( row['commitment_term']) / 30, 2)
    if row['commitment_unit'] == "Natural Life":
        return 1560.
    else:
        return 0.

data['month'] = data.apply(lambda row:month_convert(row), axis = 1)

In [ ]:
# Create column with all values in years
def year_convert(row):
    if row ['commitment_unit'] == 'Year(s)':
        return round(float(row['commitment_term']),2)
    if row["commitment_unit"] == "Months":
        return round(int(row["commitment_term"]) / 12.0, 2)
    if row["commitment_unit"] == "Weeks":
       return round(float(row['commitment_term']) / 52, 2)
    if row["commitment_unit"] == "Days":
        return round(float( row['commitment_term'])/365, 2)
    if row['commitment_unit'] == "Natural Life":
        return 130.
    else:
        return 0.

data['year'] = data.apply(lambda row:year_convert(row), axis = 1)

## Create bins for age

In [ ]:
# Delete all 'Promis Conversion' data from dataframe
data.replace(regex=['PROMIS*'], value=np.nan, inplace=True)
data.dropna(inplace=True)

In [ ]:
# Change age dtype from string to integer 
data.age_at_incident = data.age_at_incident.astype(int)

In [ ]:
# Bins to group age
bins_ranges = [0,18,24,29,39,49,59,137]
bins_names = ["<18", '18-24', '25-29', '30s', '40s', '50s', '60+']

data['age_bins'] = pd.cut(data.age_at_incident,bins_ranges,labels=bins_names)

In [ ]:
# Drop duplicates of data with current values
data.drop_duplicates(inplace=True)
data.reset_index(drop=True,inplace=True)

In [ ]:
data

## Cleaning of Charge Disposition

In [ ]:
data.charge_disposition.value_counts()

In [ ]:
# Delete data that isn't the following
data = data[
    (data.charge_disposition != 'WOWI') & 
    (data.charge_disposition != 'Superseded by Indictment') & 
    (data.charge_disposition != 'Death Suggested-Cause Abated') &
    (data.charge_disposition != 'Sexually Dangerous Person')
]

In [ ]:
# Consolidate No Guilty Findings
data.charge_disposition = data.charge_disposition.replace(to_replace=[
    'Nolle Prosecution', 
    'Case Dismissed', 
    'FNG',
    'FNG Reason Insanity',
    'FNPC',
    'SOLW',
    'Charge Vacated'
],value='No Guilty Finding')

In [ ]:
# Consolidate Finding Guilty
data.charge_disposition = data.charge_disposition.replace(to_replace=[
    'Finding Guilty - Lesser Included',
    'Finding Guilty But Mentally Ill'
], value='Finding Guilty')

In [ ]:
# Consolidate Plea of Guilty
data.charge_disposition = data.charge_disposition.replace(to_replace=[
    'Plea of Guilty - Amended Charge',
    'Plea of Guilty But Mentally Ill',
    'Plea of Guilty - Lesser Included'
], value='Plea Of Guilty')

In [ ]:
data.charge_disposition.value_counts()

# Analize new dataframe to create the database diagram

In [ ]:
data.columns

![DBD-from-quickDBD.png](../images/data_model.png)

# Create dataframe tables that will populate the database

In [ ]:
# Create participants df table
participants = data[['case_participant_id','age_at_incident','gender','race','age_bins']].drop_duplicates()

In [ ]:
courts

In [ ]:
# Create courts df table
courts = data[['court_facility', 'court_name']].drop_duplicates().reset_index(drop=True)
courts['court_id']=['1-26','6','2','5','4','3','1-4','1-DV','1-1','1-3','1-2','1-5','1-RJCC']
courts = courts[['court_id', 'court_facility', 'court_name']]

In [ ]:
# Create offense df table
offenses = data[['offense_category']].drop_duplicates().reset_index(drop=True)
offenses['offense_id'] = offenses.index + 1
offenses = offenses[['offense_id','offense_category']]

In [ ]:
# Create sentences df table
sentences = data[['sentence_type','commitment_term','commitment_unit','month', 'year']].drop_duplicates().reset_index(drop=True)
sentences['sentence_id'] = sentences.index + 1
sentences = sentences[['sentence_id','sentence_type','commitment_term','commitment_unit','month', 'year']]

In [ ]:
# Create results df table
results = data.merge(sentences,on=['sentence_type','commitment_term','commitment_unit','month','year'])
results = results.merge(offenses,on='offense_category')
results = results.merge(courts,on=['court_facility','court_name'])
results = results[['case_participant_id','court_id', 'offense_id', 'sentence_id', 'case_id', 'charge_disposition', 'length_of_case_in_days', 'disposition_charged_offense_title', 'charge_count', 'disposition_charged_class', 'sentence_date', 'incident_begin_date', 'arrest_date']]

# Create database and create the tables defined before

In [ ]:
# Create connection to database
conn = sqlite3.connect("sentencing.db")
c = conn.cursor()

In [ ]:
# Clean table if exist
pragma_fk = """PRAGMA foreign_keys = OFF;"""
drop_results = """DROP TABLE IF EXISTS results"""
drop_sentences = """DROP TABLE IF EXISTS sentences"""
drop_offenses = """DROP TABLE IF EXISTS offenses"""
drop_courts = """DROP TABLE IF EXISTS courts"""
drop_participants = """DROP TABLE IF EXISTS participants"""
clear_db = [pragma_fk,drop_results,drop_sentences,drop_offenses,drop_courts,drop_participants]
for i in clear_db:
    c.execute(i)

In [ ]:
table_participants = """
CREATE TABLE "participants" (
    "case_participant_id" bigint,
    "age_at_incident" integer,
    "gender" varchar,
    "race" varchar,
    "age_bins" varchar,
CONSTRAINT "pk_participants" PRIMARY KEY ("case_participant_id")
);
"""
c.execute(table_participants)

In [ ]:
table_courts = """
CREATE TABLE "courts" (
    "court_id" varchar,
    "court_facility" varchar,
    "court_name" varchar,
CONSTRAINT "pk_courts" PRIMARY KEY ("court_id")
);
"""
c.execute(table_courts)

In [ ]:
table_offenses = """
CREATE TABLE "offenses" (
 "offense_id" integer,
 "offense_category" varchar,
 CONSTRAINT "pk_offenses" PRIMARY KEY (
 "offense_id"
 )
);
"""
c.execute(table_offenses)

In [ ]:
table_sentences = """
CREATE TABLE "sentences" (
    "sentence_id" integer,
    "sentence_type" varchar,
    "commitment_term" float,
    "commitment_unit" varchar,
    "month" float,
    "year" float,
 CONSTRAINT "pk_sentences" PRIMARY KEY ("sentence_id")
);
"""
c.execute(table_sentences)

In [ ]:
table_results = """
CREATE TABLE "results" (
    "case_participant_id" bigint,
    "court_id" varchar,
    "offense_id" integer,
    "sentence_id" integer,
    "case_id" bigint,
    "charge_disposition" varchar,
    "length_of_case_in_days" bigint,
    "disposition_charged_offense_title" varchar,
    "charge_count" integer,
    "disposition_charged_class" varchar,
    "sentence_date" date,
    "incident_begin_date" date,
    "arrest_date" date,
    FOREIGN KEY(case_participant_id) REFERENCES participants (case_participant_id),
    FOREIGN KEY(court_id) REFERENCES courts (court_id),
    FOREIGN KEY(offense_id) REFERENCES offenses (offense_id),
    FOREIGN KEY(sentence_id) REFERENCES sentences (sentence_id)
);
"""
c.execute(table_results)

# Add data to database

In [ ]:
participants.to_sql(name='participants',con=conn,if_exists='append',index=False)
courts.to_sql(name='courts',con=conn,if_exists='append',index=False)
offenses.to_sql(name='offenses',con=conn,if_exists='append',index=False)
sentences.to_sql(name='sentences',con=conn,if_exists='append',index=False)
results.to_sql(name='results',con=conn,if_exists='append',index=False)

In [ ]:
pd.read_sql_query('Select * from results', con=conn)

In [ ]:
data.to_csv('data.csv', index=False)